# Практикум по докингу ч.1
На этом занятии разберемся, как подготовить белок и лиганды к докингу в Autodock Vina, запустить докинг и проанализировать результаты

Для начала работы нужны следующие файлы:
- `data/7e2y-receptor.pdb` – структура рецептора без лиганда
- `data/5ht1a-chembl-ki-selected.sdf` – структуры лигандов, выбранные на семинаре

### Устанавливаем необходимые библиотеки

In [ ]:
!pip install rdkit meeko

### Устанавливаем Vina

In [ ]:
!wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64
!mv vina_1.2.3_linux_x86_64 vina
!chmod 755 -R ./vina

In [1]:
alias vina ./vina

In [2]:
%vina --version

AutoDock Vina v1.2.3


### Устанавливаем OpenBabel
*Если работаете из образа `jupyter/scipy-notebook`, то отдельно в терминале нужно запустить*
``` bash
sudo apt-get update
sudo apt-get install openbabel
```
*и ввесли пароль `jupyter`. Если из гугл-колаба – просто запускаем следующую ячейку*

In [ ]:
!apt install openbabel

In [3]:
!obabel -V

Open Babel 3.1.1 -- Feb  7 2022 -- 06:51:49


## Подготовка бокса

Для того, чтобы получить координаты, необходимо:

1. Иметь установленный PyMol
2. Установить в PyMol плагин [GetBox,](https://github.com/MengwuXiao/GetBox-PyMOL-Plugin/tree/master) следуя инструкциям из репозитория
3. Загрузить в PyMol нужный белок (пусть будет человеческий альбумин 7Y2D)
4. Выбрать интересующий лиганд
5. Plugin -> Legacy Plugins -> GetBox Plugin -> Get box from selection (sele)
6. Выбрать из полученных вариантов терминале PyMol координаты для Vina

Для примера:

```
TITLE     HSA-Cu agent complex
 ExecutiveLoad-Detail: Detected mmCIF
 CmdLoad: "./7y2d.cif" loaded as "7Y2D".
 You clicked /7Y2D/H/A/IC4`1007/C17
 Selector: selection "sele" defined with 18 atoms.
*********AutoDock Vina Binding Pocket*********
--center_x 30.8 --center_y -1.1 --center_z 31.2 --size_x 14.1 --size_y 18.6 --size_z 18.0

*********AutoDock Grid Option*********
npts 37 49 48 # num. grid points in xyz
spacing 0.375 # spacing (A)
gridcenter 30.834 -1.082 31.161 # xyz-coordinates or auto

*********LeDock Binding Pocket*********
Binding pocket
23.8 37.9
-10.4 8.2
22.2 40.2

BoxCode(box_9663) = showbox 23.8, 37.9, -10.4, 8.2, 22.2, 40.2

```

Нас интересует эта строка:


```
*********AutoDock Vina Binding Pocket*********
--center_x 30.8 --center_y -1.1 --center_z 31.2 --size_x 14.1 --size_y 18.6 --size_z 18.0
```

Сохраним её значения в переменные:

In [4]:
# 7e2y
#--center_x 128.9 --center_y 125.8 --center_z 136.9 --size_x 17.4 --size_y 17.5 --size_z 15.6
center = (128.9, 125.8, 136.9)
size = (18, 18, 18)

## Подготовка рецептора
`--partialcharge gasteiger`: метод задания частичных зарядов

`-p 7.4`: pH среды

`-xr`: сгенерировать жесткий рецептор

In [5]:
!obabel data/7e2y-receptor.pdb --partialcharge gasteiger -p 7.4 -O data/7e2y-receptor.pdbqt -xr

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is data/7e2y-receptor.pdb)

1 molecule converted


## Подготорка лиганда


In [6]:
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy
from meeko import PDBQTMolecule
from meeko import RDKitMolCreate
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem.rdmolfiles import SDMolSupplier, SDWriter
from pathlib import Path

from subprocess import run

In [7]:
ligands = [
    ("VERNAKALANT", "COc1ccc(CCO[C@@H]2CCCC[C@H]2N2CC[C@@H](O)C2)cc1OC"),
    ("AZD1305", "CC(C)(C)OC(=O)NCCN1CC2CN(CCOc3ccc(C#N)cc3F)CC(C1)O2")
]

In [8]:
PREPARATOR = MoleculePreparation(
    min_ring_size=5  # Циклы > 5 становятся гибкими
)

def rdmol_to_pdbqt_string(mol):
    preparator = PREPARATOR

    ligand = Chem.rdmolops.AddHs(mol)
    rdDistGeom.EmbedMolecule(ligand)

    mol_setups = preparator.prepare(ligand)

    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            return pdbqt_string
        else:
            raise ValueError(error_msg)

In [9]:
def smi_to_lig_pdbqt(name: str, smiles: str):
    preparator = MoleculePreparation(
        min_ring_size=5 # Циклы > 5 становятся гибкими
    )
    ligand = Chem.MolFromSmiles(smiles)

    pdbqt_string = rdmol_to_pdbqt_string(ligand)

    with open(f"/content/{name}_ligand.pdbqt", "w") as ligand_pdbqt:
        ligand_pdbqt.write(pdbqt_string)

In [10]:
def sdf_to_pdbqts(sdf_path):
    pdbqt_fname_prefix = sdf_path.stem
    pdbqt_dir = sdf_path.parent / pdbqt_fname_prefix
    pdbqt_dir.mkdir(parents=False, exist_ok=True)

    with SDMolSupplier(sdf_path.resolve().absolute().as_posix()) as sdf_suppl:
        for n, mol in enumerate(sdf_suppl):
            ligand_pdbqt_string = rdmol_to_pdbqt_string(mol)
            with (pdbqt_dir / f"ligand_{n:>02}.pdbqt").open("w") as ligfile:
                ligfile.write(ligand_pdbqt_string)
    return pdbqt_dir

In [11]:
ligands_dir = sdf_to_pdbqts(Path("data/5ht1a-chembl-ki-selected.sdf"))

[10:54:09] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:09] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:09] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:10] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:10] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:10] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:10] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:10] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:54:10] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol

In [12]:
ligands_dir

PosixPath('data/5ht1a-chembl-ki-selected')

## Запускаем докинг

In [13]:
def run_vina(ligand_path, receptor_path=Path("data/7e2y-receptor.pdbqt")):
    vina_command = " ".join([
        "--receptor", f"{receptor_path.absolute().as_posix()}",
        "--ligand",   f" {ligand_path.absolute().as_posix()}",
        "--out",      f"{(ligand_path.parent / f'{ligand_path.stem}_result.pdbqt').absolute().as_posix()}",
        "--center_x", f"{center[0]}",
        "--center_y", f"{center[1]}",
        "--center_z", f"{center[2]}",
        "--size_x",   f"{size[0]}",
        "--size_y",   f"{size[1]}",
        "--size_z",   f"{size[2]}",
        "--exhaustiveness", "1",  # Можно увеличить, проверял, работает ли функция вообще
        "--num_modes", "1",
    ])
    logfile = (ligands_dir/f"{ligand_path.stem}.log").as_posix()
    !./vina {vina_command} > {logfile}

In [15]:
for ligand_path in ligands_dir.glob("*.pdbqt"):
    run_vina(ligand_path)

^C


PDBQT parsing error: Unexpected multi-MODEL tag found in flex residue or ligand PDBQT file. Use "vina_split" to split flex residues or ligands in multiple PDBQT files.

^C



KeyboardInterrupt



## Конвертируем результат

In [16]:
def pdbqt_to_sdf(result_path):
    export_command = (
        f"{result_path.absolute().as_posix()} "
        f"-o {(result_path.parent / f'{result_path.stem}.sdf').absolute().as_posix()}"
    )
    !mk_export.py {export_command}

In [17]:
for result_path in ligands_dir.glob("*_result.pdbqt"):
    pdbqt_to_sdf(result_path)

In [18]:
from rdkit.Chem.rdmolfiles import SDMolSupplier, SDWriter
from pathlib import Path

with SDWriter("5ht1a-selected-docking.sdf") as dst:
    for result_path in ligands_dir.glob("*_result.sdf"):
        with SDMolSupplier(result_path.absolute().as_posix()) as src:
            for lig in src:
                dst.write(lig)